In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime as dt
import datetime
from sklearn.metrics import roc_auc_score,average_precision_score,recall_score
import letor_metrics as lm
import os
import csv
import re
from collections import Counter
import random

import theano
import DataParser

from keras.layers.core import Activation, Dense, Merge, Lambda
from keras import backend as K
from keras.models import Sequential,Model
from keras.layers import Dense, LSTM, Flatten,SimpleRNN,Embedding,Input, merge,Reshape
from keras.preprocessing import sequence
from keras.optimizers import Adam,SGD
from keras.regularizers import l2, activity_l2,l1


pd.set_option('display.precision', 16)
theano.config.openmp = True
OMP_NUM_THREADS=4 


#params
latent_dim = 10

In [2]:
mode = "day"
WINDOW = 7

df, df_test,uNum, vNum = DataParser.readCheckinData('../datasets/gowallaTrain', '../datasets/gowallaTest')
# convert to sparse matrix for each time window
tuv = DataParser.sparseDataset(df, mode, WINDOW, uNum, vNum)
tuv_test = DataParser.sparseDataset(df_test, mode, WINDOW, uNum, vNum)
# pos/neg samples
pos,neg = DataParser.generate_train_pairs(tuv)

In [8]:
scores = {"p": [], "ndcg": [], "auc": [], "map": []}
for t in range(WINDOW):
    grnd = tuv_test[t]
    for u in range(uNum):
        if np.all(true[u]==0):
            continue
        
        scores['auc'].append(roc_auc_score(grnd[u], pred[u].reshape((vNum,))))
        scores['map'].append(average_precision_score(grnd[u], pred[u].reshape((vNum,))))
        scores['p'].append(lm.ranking_precision_score(grnd[u], pred[u].reshape((vNum,))))
        scores['ndcg'].append(lm.ndcg_score(grnd[u], pred[u].reshape((vNum,))))

print('AUC, P@10, NDCG, MAP %s %s %s %s' % sum(scores['auc']) / len(scores['auc']), sum(scores['p']) / len(scores['p']), sum(scores['ndcg']) / len(scores['ndcg']), sum(scores['map']) / len(scores['map']))


0
1
2
3
4
5
6


In [81]:
def rnn_triplet_loss(X):

    positive_item_latent, negative_item_latent, user_latent, pos_user_temporal_latent,neg_user_temporal_latent, pos_user_temporal_bw_embedding,neg_user_temporal_bw_embedding = X
    
    
#   TODO if user_temporal_latent[0][0] == [0,,,,,0] ignore, else sum all loss

    loss = 0
#     for t in range(WINDOW):
    loss = (1 - K.sigmoid(
            K.sum(( pos_user_temporal_bw_embedding + pos_user_temporal_latent + user_latent) * positive_item_latent, axis=-1, keepdims=True) -
            K.sum(( neg_user_temporal_bw_embedding + neg_user_temporal_latent + user_latent) * negative_item_latent, axis=-1, keepdims=True)))
    
    
    
    return loss



def rnn_predict(model,user_temporal_embedding_layer,uid, pids, t):

    user_vector = model.get_layer('user_embedding').get_weights()[0][uid]
    item_matrix = model.get_layer('item_embedding').get_weights()[0][pids]
    user_temporal_vector = user_temporal_embedding_layer.predict(pos[uid].reshape(1,WINDOW))[0][t]
    user_temporal_bw_vector = user_temporal_embedding_layer_bw.predict(pos[uid].reshape(1,WINDOW))[0][t]


    scores = (np.dot(user_vector+user_temporal_vector+user_temporal_bw_vector,
                     item_matrix.T))

    return scores






# Shared embedding layer for positive and negative items
item_embedding_layer = Embedding(
    vNum+1, latent_dim, name='item_embedding', input_length=WINDOW)

lstm = SimpleRNN(latent_dim, return_sequences=True)

lstm_bw = SimpleRNN(latent_dim, return_sequences=True, go_backwards= True)

#     user_temporal_embedding_layer = lstm
user_temporal_embedding_layer = Sequential(name="user_temporal_embedding")
user_temporal_embedding_layer.add(item_embedding_layer)
user_temporal_embedding_layer.add(lstm)

user_temporal_embedding_layer_bw = Sequential(name="user_temporal_bw_embedding")
user_temporal_embedding_layer_bw.add(item_embedding_layer)
user_temporal_embedding_layer_bw.add(lstm_bw)


positive_item_input = Input((WINDOW, ), name='positive_item_input')
negative_item_input = Input((WINDOW, ), name='negative_item_input')
user_input = Input((WINDOW, ), name='user_input')



positive_item_embedding = Flatten()(item_embedding_layer(positive_item_input))
negative_item_embedding = Flatten()(item_embedding_layer(negative_item_input))


#     positive_item_embedding = (item_embedding_layer(positive_item_input))
#     negative_item_embedding = (item_embedding_layer(negative_item_input))


user_embedding = Flatten()(Embedding(
    uNum, latent_dim, name='user_embedding', input_length=1)(
        user_input))


pos_user_temporal_embedding = Flatten()(user_temporal_embedding_layer(positive_item_input))
neg_user_temporal_embedding = Flatten()(user_temporal_embedding_layer(negative_item_input))

pos_user_temporal_bw_embedding = Flatten()(user_temporal_embedding_layer_bw(positive_item_input))
neg_user_temporal_bw_embedding = Flatten()(user_temporal_embedding_layer_bw(negative_item_input))

loss = merge(
    [positive_item_embedding, negative_item_embedding, user_embedding,pos_user_temporal_embedding,neg_user_temporal_embedding,pos_user_temporal_bw_embedding,neg_user_temporal_bw_embedding],
    mode=rnn_triplet_loss,
    name='loss',
    output_shape=(WINDOW, ))

rnn = Model(
    input=[positive_item_input, negative_item_input, user_input],
    output=[loss])
rnn.compile(loss=identity_loss, optimizer=Adam())
# rnn.compile(loss=identity_loss, optimizer=SGD(lr=0.0001))
    

u_input = []
for u in range(uNum):
    ut = []
    for i in range(WINDOW):
        ut.append(u)
    u_input.append(ut)
u_input = np.array(u_input)

for epoch in range(100):
    
    print(datetime.datetime.now())
    print('Epoch %s' % epoch)
    
    
    
    
    X = {
        'positive_item_input': pos,
        'negative_item_input': neg,
        'user_input' : u_input
    }

    rnn.fit(X,
              np.ones(uNum),
              batch_size=uNum,
              nb_epoch=1,
              verbose=1,
              shuffle=True)
#     print('AUC %s' % full_auc(rnn,user_temporal_embedding_layer, tuv_test))

2017-03-20 15:18:11.531871
Epoch 0


KeyboardInterrupt: 

In [79]:
"""
Triplet loss network example for recommenders
"""

from __future__ import print_function

import numpy as np
random.seed(1234)
import scipy.sparse as sp
import pandas as pd
from keras import backend as K
from keras.models import Model
from keras.layers import Embedding, Flatten, Input, merge
from keras.optimizers import Adam,SGD
import metrics
from keras.regularizers import l2, activity_l2
import datetime


def identity_loss(y_true, y_pred):

    return K.mean(y_pred - 0 * y_true)


# def identity_loss(y_true, y_pred):

#     return K.mean(y_pred - 0 * y_true)

def bpr_triplet_loss(X):

    positive_item_latent, negative_item_latent, user_latent = X
    
    reg = 0.001 *(K.sum(user_latent**2, axis=-1, keepdims=True) + K.sum(positive_item_latent**2, axis=-1, keepdims=True) + K.sum(negative_item_latent**2, axis=-1, keepdims=True))
    
    
    # BPR loss
    loss = 1 - K.sigmoid(
        K.sum(user_latent * positive_item_latent, axis=-1, keepdims=True) -
        K.sum(user_latent * negative_item_latent, axis=-1, keepdims=True)) - reg

#     loss = K.log(K.sigmoid(K.dot(user_latent, K.transpose(positive_item_latent)) - K.dot(user_latent, K.transpose(negative_item_latent)))) - reg
#     loss = 1 - K.sigmoid(
#             K.sum(user_latent * positive_item_latent, axis=-1, keepdims=True) -
#             K.sum(user_latent * negative_item_latent, axis=-1, keepdims=True))

    
#     loss = K.sum(K.log(K.sigmoid(user_latent * positive_item_latent - user_latent * negative_item_latent)), axis=-1, keepdims=True) + reg
    
    return loss

# BPR
def bpr_predict(model, uid, pids):

    user_vector = model.get_layer('user_embedding').get_weights()[0][uid]
    item_matrix = model.get_layer('item_embedding').get_weights()[0][pids]
    
    scores = (np.dot(user_vector,
                     item_matrix.T))

    return scores


def build_model(uNum, vNum, latent_dim):

        # Shared embedding layer for positive and negative items
    item_embedding_layer = Embedding(
        vNum, latent_dim, name='item_embedding', input_length=WINDOW)
    
#     ,W_regularizer=l2(0.01)


    positive_item_input = Input((WINDOW, ), name='positive_item_input')
    negative_item_input = Input((WINDOW, ), name='negative_item_input')
    user_input = Input((WINDOW, ), name='user_input')


    positive_item_embedding = Flatten()(item_embedding_layer(positive_item_input))
    negative_item_embedding = Flatten()(item_embedding_layer(negative_item_input))


    user_embedding = Flatten()(Embedding(
        uNum, latent_dim, name='user_embedding', input_length=1)(
            user_input))



    loss = merge(
        [positive_item_embedding, negative_item_embedding, user_embedding],
        mode=bpr_triplet_loss,
        name='loss',
        output_shape=(WINDOW, ))

    model = Model(
        input=[positive_item_input, negative_item_input, user_input],
        output=[loss])
    model.compile(loss=identity_loss, optimizer=SGD())
#     model.compile(loss=identity_loss, optimizer=Adam())
    return model


bpr = build_model(uNum+1, vNum+1, latent_dim)
print(datetime.datetime.now())
u_input = []
for u in range(uNum):
    ut = []
    for i in range(WINDOW):
        ut.append(u)
    u_input.append(ut)
u_input = np.array(u_input)

for epoch in range(100):
    
#     print('Epoch %s' % epoch)

#     pos,neg = DataParser.generate_train_pairs(tuv)


    
    X = {
        'positive_item_input': pos,
        'negative_item_input': neg,
        'user_input' : u_input
    }

    bpr.fit(X,
              np.ones(uNum),
              batch_size=uNum,
              nb_epoch=1,
              verbose=1,
              shuffle=True)
print('AUC, P@10, NDCG, MAP %s %s %s %s' % evaluation(tuv_test, "bpr"))

2017-03-20 13:27:31.981487
Epoch 1/1
34796/34796 [==============================] - 0s - loss: 0.4998
Epoch 1/1
34796/34796 [==============================] - 0s - loss: 0.4998
Epoch 1/1
34796/34796 [==============================] - 0s - loss: 0.4998
Epoch 1/1
34796/34796 [==============================] - 0s - loss: 0.4998
Epoch 1/1
34796/34796 [==============================] - 0s - loss: 0.4998
Epoch 1/1
34796/34796 [==============================] - 0s - loss: 0.4998
Epoch 1/1
34796/34796 [==============================] - 0s - loss: 0.4998
Epoch 1/1
34796/34796 [==============================] - 0s - loss: 0.4998
Epoch 1/1
34796/34796 [==============================] - 0s - loss: 0.4998
Epoch 1/1
34796/34796 [==============================] - 0s - loss: 0.4998
Epoch 1/1
34796/34796 [==============================] - 0s - loss: 0.4998
Epoch 1/1
34796/34796 [==============================] - 0s - loss: 0.4998
Epoch 1/1
34796/34796 [==============================] - 0s - loss: 0.499

In [63]:
df_test = pd.read_csv(filename, sep=separater, names=names)

NameError: name 'filename' is not defined

In [ ]:
def build_tf_model(uNum, vNum, WINDOW, latent_dim):

        # Shared embedding layer for positive and negative items
    item_embedding_layer = Embedding(
        vNum, latent_dim, name='item_embedding', input_length=WINDOW)


    positive_item_input = Input((WINDOW, ), name='positive_item_input')
    negative_item_input = Input((WINDOW, ), name='negative_item_input')
    user_input = Input((WINDOW, ), name='user_input')


    positive_item_embedding = Flatten()(item_embedding_layer(positive_item_input))
    negative_item_embedding = Flatten()(item_embedding_layer(negative_item_input))


    user_embedding = Flatten()(Embedding(
        uNum, latent_dim, name='user_embedding', input_length=1)(
            user_input))



    loss = merge(
        [positive_item_embedding, negative_item_embedding, user_embedding],
        mode=bpr_triplet_loss,
        name='loss',
        output_shape=(WINDOW, ))

    model = Model(
        input=[positive_item_input, negative_item_input, user_input],
        output=[loss])
    model.compile(loss=identity_loss, optimizer=Adam())

    return model

In [163]:
import numpy as np
from keras.layers import Embedding, Reshape, Merge, Dropout, Dense
from keras.models import Sequential

class MF(Sequential):

    def __init__(self, n_users, m_items, k_factors, **kwargs):
        P = Sequential()
        P.add(Embedding(n_users, k_factors, input_length=1, W_regularizer=l2(0.001)))
        P.add(Reshape((k_factors,)))
        Q = Sequential()
        Q.add(Embedding(m_items, k_factors, input_length=1, W_regularizer=l2(0.001)))
        Q.add(Reshape((k_factors,)))
        super(CFModel, self).__init__(**kwargs)
        self.add(Merge([P, Q], mode='dot', dot_axes=1))

    def rate(self, user_id, item_id):
        return self.predict([np.array([user_id]), np.array([item_id])])[0][0]
    
    def rank(self, user_id, venues):
        u = np.random.rand(vNum)
        u.fill(user_id)
        return self.predict([u, venues])
    

In [ ]:
tf = TF(uNum, vNum, WINDOW, latent_dim)
tf.compile(loss='mse', optimizer=Adam())
tf.fit([df.uid.values, df.vid.values, df.day.values], np.ones(len(df.uid.values)), verbose=1, nb_epoch=100, batch_size=100000)

In [164]:
mf = CFModel(uNum, vNum, latent_dim)
mf.compile(loss='mse', optimizer=Adam())
mf.fit([df.uid.values, df.vid.values], np.ones(len(df.uid.values)), verbose=0, nb_epoch=100, batch_size=100000)
# mf.predict

In [272]:
def evaluation(ground_truth, mode):

    scores = {"p":[], "ndcg":[], "auc":[], "map":[] }


    for t in range(WINDOW):

        ground = ground_truth[t].tocsr()

        no_users, no_items = ground.shape

        pid_array = np.arange(no_items, dtype=np.int32)

        stop = 0
        for user_id, row in enumerate(ground):
            if mode == "mf":
                usr = np.empty(vNum)
                usr.fill(user_id)
                predictions = mf.predict([usr, pid_array])
            if mode == "rnn":
                predictions = rnn_predict(rnn,user_temporal_embedding_layer, user_id, pid_array,t)
            if mode == "bpr":
                predictions = bpr_predict(bpr, user_id,pid_array)
                
            if mode == "tf":
                usr = np.empty(vNum)
                time = np.empty(vNum)
                usr.fill(user_id)
                time.fill(t)
                predictions = tf.predict([usr, pid_array, time])
                



            true_pids = row.indices[row.data == 1]

            grnd = np.zeros(no_items, dtype=np.int32)
            grnd[true_pids] = 1
            

            if len(true_pids) > 0:
                scores['auc'].append(roc_auc_score(grnd, predictions.reshape((vNum,))))
                scores['map'].append(average_precision_score(grnd, predictions.reshape((vNum,))))
                scores['p'].append(lm.ranking_precision_score(grnd, predictions.reshape((vNum,))))
                scores['ndcg'].append(lm.ndcg_score(grnd, predictions.reshape((vNum,))))
            
            stop = stop +1
#             print(stop)
            if stop == 100:
                break
                


#     print(datetime.datetime.now())
    return sum(scores['auc']) / len(scores['auc']),sum(scores['p']) / len(scores['p']),sum(scores['ndcg']) / len(scores['ndcg']),sum(scores['map']) / len(scores['map'])

In [288]:
# SGD with reg
evaluation(tuv_test, "bpr")

(0.4996847146681565,
 0.0012345679012345679,
 0.00086403488602334352,
 0.00031593478448919439)

In [296]:
# Adam with my reg
evaluation(tuv_test, "bpr")

(0.91530710036097773,
 0.049149029982363319,
 0.047939434276729462,
 0.028098317319904607)

In [273]:
evaluation(tuv_test, "bpr")

(0.89528732135303157,
 0.050240299823633157,
 0.050797409636055751,
 0.033277904978059665)

In [161]:
evaluation(tuv_test, "tf")

(0.89774425125291513,
 0.015401234567901234,
 0.012973438569239316,
 0.011945292287429835)

In [165]:
evaluation(tuv_test, "mf")

(0.8951780459764842, 0.0, 0.0, 0.0014938904625095384)

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
y_true = np.array(['cat', 'dog', 'pig', 'cat', 'dog', 'pig'])
y_pred = np.array(['cat', 'pig', 'dog', 'cat', 'cat', 'dog'])
precision_recall_fscore_support(y_true, y_pred, average='macro')

In [ ]:
a = df['hour'].apply(lambda x: x%6)

In [ ]:
df6 = df
df6_test = df_test

df6.loc[:,'hour'] %= 6
df6_test.loc[:,'hour'] %= 6

In [111]:
class TF(Sequential):
    def __init__(self, uNum, vNum, latent_dim, WINDOW, **kwargs):

        def dot3D(x):
            P, Q, T = x;
            return K.sum(P * Q * T, axis=-1, keepdims=True);

        P = Sequential()
        P.add(Embedding(uNum, latent_dim, input_length=1, W_regularizer=l2(0.001)))
        P.add(Reshape((latent_dim,)))
        Q = Sequential()
        Q.add(Embedding(vNum, latent_dim, input_length=1, W_regularizer=l2(0.001)))
        Q.add(Reshape((latent_dim,)))
        T = Sequential()
        T.add(Embedding(WINDOW, latent_dim, input_length=1, W_regularizer=l2(0.001)))
        T.add(Reshape((latent_dim,)))

        super(TF, self).__init__(**kwargs)
        self.add(Merge([P, Q, T], mode=dot3D, output_shape=[1,]))
    

In [117]:
tf = TF(uNum, vNum, latent_dim, WINDOW)
tf.compile(loss='mse', optimizer=Adam())

In [120]:
tf.fit([df.uid, df.vid, df.day], np.ones(len(df)), nb_epoch=10, verbose=0, batch_size=100000)

In [144]:
tf.predict([u, np.arange(vNum), t])

array([[ 0.80231273],
       [ 0.99178833],
       [ 0.97638017],
       ..., 
       [ 0.00491236],
       [ 0.05006572],
       [ 0.45416552]], dtype=float32)

In [189]:
uv = sp.lil_matrix((uNum, vNum), dtype=np.int32)
for u,v in zip(df.uid.values, df.vid.values):
    uv[u,v] = 1

In [201]:
pair_u = []
pair_i = []
pair_j = []
for u in range(uNum):
    visited = uv[u].tocoo().col
    for i in visited:
        j = np.random.randint(vNum)
        while j in visited:
            j = np.random.randint(vNum)
        pair_u.append(u)
        pair_i.append(i)
        pair_j.append(j)
        
pair_u = np.array(pair_u)
pair_i = np.array(pair_i)
pair_j = np.array(pair_j)

In [293]:

def identity_loss(y_true, y_pred):

    return K.mean(y_pred - 0 * y_true)


def bpr_triplet_loss(X):

    positive_item_latent, negative_item_latent, user_latent = X
    
    reg = 0.001 * (K.sum(user_latent ** 2, axis=-1, keepdims=True) + K.sum(positive_item_latent ** 2, axis=-1,
                                                                                   keepdims=True) + K.sum(
                negative_item_latent ** 2, axis=-1, keepdims=True))

    # BPR loss
    loss = 1.0 - K.sigmoid(
        K.sum(user_latent * positive_item_latent, axis=-1, keepdims=True) -
        K.sum(user_latent * negative_item_latent, axis=-1, keepdims=True)) + reg

    return loss

def bpr_predict(model, uid, vids):

    user_vector = model.get_layer('user_embedding').get_weights()[0][uid]
    item_matrix = model.get_layer('item_embedding').get_weights()[0][vids]

    scores = (np.dot(user_vector,
                         item_matrix.T))

    return scores


def build_model(num_users, num_items, latent_dim):

    positive_item_input = Input((1, ), name='positive_item_input')
    negative_item_input = Input((1, ), name='negative_item_input')

    # Shared embedding layer for positive and negative items
    item_embedding_layer = Embedding(
        num_items, latent_dim, name='item_embedding', input_length=1)

    user_input = Input((1, ), name='user_input')

    positive_item_embedding = Flatten()(item_embedding_layer(
        positive_item_input))
    negative_item_embedding = Flatten()(item_embedding_layer(
        negative_item_input))
    user_embedding = Flatten()(Embedding(
        num_users, latent_dim, name='user_embedding', input_length=1)(
            user_input))

    loss = merge(
        [positive_item_embedding, negative_item_embedding, user_embedding],
        mode=bpr_triplet_loss,
        name='loss',
        output_shape=(1, ))

    model = Model(
        input=[positive_item_input, negative_item_input, user_input],
        output=loss)
    model.compile(loss=identity_loss, optimizer=Adam())

    return model

In [294]:
bpr = build_model(uNum+1, vNum+1, 10)

In [295]:
        
X = {
    'positive_item_input': pair_i,
    'negative_item_input': pair_j,
    'user_input' : pair_u
}

bpr.fit(X,
          np.ones(pair_i.shape[0]),
          batch_size=100000,
          nb_epoch=100,
          verbose=1,
          shuffle=True)



Epoch 1/100
974739/974739 [==============================] - 1s - loss: 0.5000     
Epoch 2/100
974739/974739 [==============================] - 1s - loss: 0.4999     
Epoch 3/100
974739/974739 [==============================] - 1s - loss: 0.4999     
Epoch 4/100
974739/974739 [==============================] - 1s - loss: 0.4998     
Epoch 5/100
974739/974739 [==============================] - 1s - loss: 0.4996     
Epoch 6/100
974739/974739 [==============================] - 1s - loss: 0.4995     
Epoch 7/100
974739/974739 [==============================] - 1s - loss: 0.4993     
Epoch 8/100
974739/974739 [==============================] - 1s - loss: 0.4991     
Epoch 9/100
974739/974739 [==============================] - 1s - loss: 0.4988     
Epoch 10/100
974739/974739 [==============================] - 1s - loss: 0.4984     
Epoch 11/100
974739/974739 [==============================] - 1s - loss: 0.4978     
Epoch 12/100
974739/974739 [==============================] - 1s - loss: 0

In [ ]:
pair_u = []
pair_i = []
pair_j = []
for u in range(uNum):
    visited = uv[u].tocoo().col
    for i in visited:
        j = np.random.randint(vNum)
        while j in visited:
            j = np.random.randint(vNum)
        pair_u.append(u)
        pair_i.append(i)
        pair_j.append(j)
        
pair_u = np.array(pair_u)
pair_i = np.array(pair_i)
pair_j = np.array(pair_j)

In [364]:
# for u,v,t in zip(df.uid, df.vid, df.hour):
#     print(u,v,t)
#     break
rnn_pair_u = []
rnn_pair_i = []
rnn_pair_j = []
rnn_last_pair = []

def sampleNeg(visited, vNum):
    j = np.random.randint(vNum)
    while j in visited:
        j = np.random.randint(vNum)
    return j

n = 0
for u in range(uNum):
    for t in range(WINDOW):
        visited = tuv[t][u].tocoo().col
        for i in visited:
            pos = np.zeros(WINDOW)
            neg = np.zeros(WINDOW)
            pos[t] = i
            neg[t] = sampleNeg(visited, vNum)
            
            for t2 in range(WINDOW):
                if t == t2:
                    continue
                
                visited2 = tuv[t2][u].tocoo().col
                if(len(visited2) > 0):
                    pos[t2] = random.choice(visited2)
                    neg[t2] = sampleNeg(visited2, vNum)

            u_ = np.zeros(WINDOW)
            u_.fill(u)
            rnn_pair_u.append(u_)
            rnn_pair_i.append(pos)
            rnn_pair_j.append(neg)
        rnn_last_pair.append(pos)
        
    n = n +1
    if n ==2:
        break
        
rnn_pair_u = np.array(rnn_pair_u)
rnn_pair_i = np.array(rnn_pair_i)
rnn_pair_j = np.array(rnn_pair_j)
rnn_last_pair = np.array(rnn_last_pair)

print(rnn_pair_i)
print(rnn_last_pair)
print(rnn_last_pair.reshape((2,7,7)))

[[   1.   76.    6. ...,   38.   51.   62.]
 [   6.   59.   69. ...,   55.   97.   37.]
 [  32.   71.   69. ...,   41.   77.   56.]
 ..., 
 [ 245.  293.  286. ...,  438.  416.  413.]
 [ 299.  236.  120. ...,  434.  417.  414.]
 [ 221.  235.  121. ...,  205.  325.  415.]]
[[ 107.    6.   87.   94.    1.   98.   84.]
 [   1.  106.   54.   99.    1.   38.   84.]
 [  56.   59.  105.  100.    1.   39.   92.]
 [  61.   76.  104.  111.    6.   86.  113.]
 [  55.   68.   39.    7.   91.   39.   62.]
 [  81.   60.   78.   64.   77.  116.   84.]
 [  56.   76.   54.  111.   91.  110.  113.]
 [ 405.  294.  286.  302.  370.  425.  255.]
 [ 118.  404.  286.  399.  354.  261.  406.]
 [ 295.  127.  462.  456.  112.  361.  155.]
 [ 150.  234.  288.  459.  283.  342.  415.]
 [ 239.  235.  444.  121.  454.  345.  153.]
 [ 298.  291.  137.  401.  134.  432.  259.]
 [ 221.  235.  121.  315.  205.  325.  415.]]
[[[ 107.    6.   87.   94.    1.   98.   84.]
  [   1.  106.   54.   99.    1.   38.   84.]
  [  

In [366]:
num_samples = 0
for t in range(WINDOW):
    num_samples = num_samples + len(tuv[t])
print(num_samples)

TypeError: sparse matrix length is ambiguous; use getnnz() or shape[0]

In [380]:
from scipy.sparse import csr_matrix

tuv[0].nonzero()[0].shape[0]

149590